# Doing Fun Things with PyRosetta
Today we're going to talk about why interacting with PyRosetta though Jupyter Notebooks and PyMOL is a good idea. Yay!
First things first, we're going to import and initialize Pyrosetta. This is always the first thing we do.

In [10]:
import pyrosetta
import pyrosetta.rosetta as rosetta

pyrosetta.init()

Found rosetta database at: /usr/local/lib/python3.6/site-packages/pyrosetta/database; using it....
PyRosetta-4 2017 [Rosetta devel 2018.26.post.dev+419.master.c869d56abb7c869d56abb713ec2f8552048670acca3247a5423 2018-07-09T18:12:08] retrieved from: https://github.com/RosettaCommons/main.git
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.

core.init: Checking for fconfig files in pwd and ./rosetta/flags

core.init: Rosetta version: 2018.26.post.dev+419.master.c869d56abb7 c869d56abb713ec2f8552048670acca3247a5423 https://github.com/RosettaCommons/main.git 2018-07-09T18:12:08
core.init: command: PyRosetta -ex1 -ex2aro -database /usr/local/lib/python3.6/site-packages/pyrosetta/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=1910382781 seed_offset=0 real_seed=1910382781
core.init.random: RandomGenerator:init: Normal mode, seed=1910382781 RG_type=mt19937


Great! Now that we have initialized PyRosetta, we can start working with proteins. One of the major formats for storing information about proteins is a PDB file. *Blah blah blah coordinates* Let's load a fun example of a protein that is interesting for some reason.

In [11]:
my_pose = rosetta.core.import_pose.pose_from_file('Assets/Demo/1o3d.pdb')

core.import_pose.import_pose: File 'Assets/Demo/1o3d.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 1 in file Assets/Demo/1o3d.pdb. Best match rsd_type:  ILE:NtermProteinFull
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 1 atoms at position 40 in file Assets/Demo/1o3d.pdb. Best match rsd_type:  HIS
core.conformation.Conformation: Found disulfide between residues 7 137
core.conformation.Conformation: current variant for 7 CYS
core.conformation.Conformation: current variant for 137 CYS
core.conformation.Conformation: current variant for 7 CYD
core.conformation.Conformation: current variant for 137 CYD
core.conformation.Conformation: Found disulfide between residues 25 41
core.conformation.Conformation: current variant for 25 CYS
core.conformation.Conformation: current variant for 41 CYS
core.conformation.Conformation: current variant for 25 CYD
core.conformation.Conformation: current 

Rosetta parses the PDB file and converts it into a representation called a "pose" that consists of fold trees, interaction graphs, residues, atoms, and more. Let's take a look at this protein in PyMOL. 

Navigate to where you've installed PyRosetta on your computer and find the file called `PyMOL-RosettaServer.py`. The path should be `<Base path>/PyRosetta/src/PyMOL-RosettaServer.py`. Open up PyMOL and enter the command `run path/to/PyMOL-RosettaServer.py`. This will open a connection between Rosetta and PyMOL so that they can communicate.

This is what it should look like:
![](Assets/Demo/RunPyMOLServer.png)

Once the connection is established, we can pass a pose to PyMOL.

In [12]:
pymover = rosetta.protocols.moves.PyMOLMover()
pymover.apply(my_pose)

pyobs = rosetta.protocols.moves.PyMOLObserver()
pyobs.attach(my_pose)

You should see the pose in Pymol now!

![](Assets/Demo/1o3d_import.png)

This oberver will allow use to watch what Rosetta is doing to the pose in real time.

## Actually Doing Seomthing in PyRosetta
Now that we can import a pose into PyRosetta and view it in PyMOL, let's try doing a simple procedure. First, let's see how good Rosetta thinks this sturucture is.

In [13]:
scorefxn = rosetta.core.scoring.get_score_function()
scorefxn(my_pose)

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015


-337.8313807680202

That's pretty good! In Rosetta, lower energies are better. Let's get a more granular view and break down where these energies are coming from.

In [14]:
pymover.update_energy(True)
pymover.send_energy(my_pose, rosetta.core.scoring.total_score)
pymover.label_energy(my_pose, 'fa_rep')
pymover.apply(my_pose)